# Geospatial Transforms

Instructors: [Tyler Sutterley](mailto:tsutterl@uw.edu), [Hannah Besso](mailto:bessoh2@uw.edu) and [Scott Henderson](mailto:scottyh@uw.edu), some material adapted from [David Shean](mailto:dshean@uw.edu)

```{admonition} Learning Objectives
- Review fundamental concepts of geospatial coordinate reference systems (CRS)
- Learn how to access CRS metadata
- Learn basic coordinate transformations relevant to ICESat-2
```

## Introduction

ICESat-2 elevations are spatial point data. Spatial data contains information about _where_ on the surface of the Earth a certain feature is located, and there are many different ways to define this location. While this seems straightforward, two main characteristics of the Earth make defining locations difficult: 

1) Earth is 3-dimensional (working with spatial data would be a lot easier if the world were flat)!

2) Paper maps and computer screens are flat, which causes issues when we try to use them to display rounded shapes (like the Earth's surface). Making things even more difficult, _the irregular shape of the Earth means there is no one perfect model of its surface on which we could place our spatial data points_! Instead, we're left with many models of the Earth's surface that are optimized for different locations and purposes. 

In [ ]:
import geopandas as gpd
from IPython.display import Image
import matplotlib.patches
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import netCDF4
import numpy as np
import os
import osgeo.gdal, osgeo.osr
import pyproj
import rasterio
import rasterio.features
import rasterio.warp
import re
import warnings
# turn off warnings
warnings.filterwarnings('ignore')

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Let's Start by Making a Map


**Q: Why would we use maps to display geographic data?**

### Geopandas vector data

We'll use geopandas build-in [Natural Earth dataset](https://www.naturalearthdata.com) to load polygons of world countries

In [ ]:
# Load a dataset consisting of land polygons
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.head()

In [ ]:
fig,ax1 = plt.subplots(num=1, figsize=(10,4.55))
minlon,maxlon,minlat,maxlat = (-180,180,-90,90)

world.plot(ax=ax1, color='0.8', edgecolor='none')

# set x and y limits
ax1.set_xlim(minlon,maxlon)
ax1.set_ylim(minlat,maxlat)
ax1.set_aspect('equal', adjustable='box')

# add x and y labels
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')

# adjust subplot positioning and show
fig.subplots_adjust(left=0.06,right=0.98,bottom=0.08,top=0.98)

**Q: So how did we just display a rounded shape on a flat surface?**

Geographic Coordinate Systems
-----------------------------
Locations on Earth are usually specified in a geographic coordinate system consisting of
- _Longitude_ specifies the angle east and west from the Prime Meridian (102 meters east of the Royal Observatory at Greenwich)
- _Latitude_ specifies the angle north and south from the Equator

The map above _projects_ geographic data from the Earth's 3-dimensional geometry on to a flat surface.  [The three common types of projections are _cylindric_, _conic_ and _planar_](https://courses.washington.edu/gis250/lessons/projection/).  Each type is a different way of flattening the Earth's geometry into 2-dimensional space.

This map is in an _Equirectangular Projection_ (Plate Carrée), where latitude and longitude are equally spaced.  Equirectangular is cylindrical projection, which has benefits as latitudes and longitudes form straight lines.  

```{warning}
While simple conceptually, this projection distorts both _shape_ and _distance_, particularly at higher latitudes! So it is not a great choice for data analysis.
```

To illustrate distortion on this map, we've colored the normalized grid area at different latitudes below:

In [ ]:
fig,ax1 = plt.subplots(num=1, figsize=(10.375,5.0))
minlon,maxlon,minlat,maxlat = (-180,180,-90,90)
dlon,dlat = (1.0,1.0)
longitude = np.arange(minlon,maxlon+dlon,dlon)
latitude = np.arange(minlat,maxlat+dlat,dlat)

# calculate and plot grid area
gridlon,gridlat = np.meshgrid(longitude, latitude)
im = ax1.imshow(np.cos(gridlat*np.pi/180.0),
    extent=(minlon,maxlon,minlat,maxlat), 
    interpolation='nearest',
    cmap=plt.cm.get_cmap('plasma'),
    origin='lower')

# add coastlines
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.plot(ax=ax1, color='none', edgecolor='black')

# set x and y limits
ax1.set_xlim(minlon,maxlon)
ax1.set_ylim(minlat,maxlat)
ax1.set_aspect('equal', adjustable='box')

# add x and y labels
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')

# add colorbar
cbar = plt.colorbar(im, cax=fig.add_axes([0.92, 0.08, 0.025, 0.90]))
cbar.ax.set_ylabel('Normalized Grid Area')
cbar.solids.set_rasterized(True)

# adjust subplot and show
fig.subplots_adjust(left=0.06,right=0.9,bottom=0.08,top=0.98)

### The Components of a Coordinate Reference System (CRS):

* **Projection Information:** the mathematical equation used to flatten objects that are on a round surface (e.g. the earth) so you can view them on a flat surface (e.g. your computer screens or a paper map).
* **Coordinate System:** the X, Y, and Z grid upon which your data is overlaid and how you define where a point is located in space.
* **Horizontal and vertical units:** The units used to define the grid along the x, y (and z) axis.
* **Datum:** A modeled version of the shape of the earth which defines the origin used to place the coordinate system in space. 

☝️Borrowed from [Earth Data Science Coursework](https://www.earthdatascience.org/courses/use-data-open-source-python/intro-vector-data-python/spatial-data-vector-shapefiles/intro-to-coordinate-reference-systems-python/)

### Notes on Projections

* _There is no perfect projection for all purposes_

* Not all maps are good for ocean or land navigation

* Not all projections are good for polar mapping

* Every projection will distort either shape, area, distance or direction:
     * _conformal_ projections minimize distortion in shape
     * _equal-area_ projections minimize distortion in area
     * _equidistant_ projections minimize distortion in distance
     * _true-direction_ projections minimize distortion in direction

While there are [projections that are better suited for specific purposes](https://pubs.usgs.gov/gip/70047422/report.pdf), [choosing a map projection](https://pubs.usgs.gov/pp/1395/report.pdf) is a bit of an art 🦋

[![](https://imgs.xkcd.com/comics/map_projections.png)](https://xkcd.com/977/)

**Q: What is your favorite projection?** 🌎

**Q: What projections do you use in your research?** 🌏

#### Determine your data's CRS

Using [geopandas](https://geopandas.org), we can get CRS information about our data:

In [ ]:
world.crs

**EPSG:** European Petroleum Survey Group. Back in the day, this group created codes to standardize how different reference systems were referred to, and now EPSG codes are widely used in geospatial work! There are several websites that let you navigate the entire EPSG database: https://epsg.io/4326

_If CRS metadata on any products isn't included within the data product, make sure it's in the right projection and datum. Often metadata reports or readme files will provide this information._

#### Reproject your data

And we can also reproject it fairly easily, as long as our vertical reference (our ellipsoid or geoid) stays the same. For example, let's use [epsg:3031](https://epsg.io/3031), which uses the same Datum, but is otherwise very different.

* Remember: "The projection is the mathematical equation used to flatten objects that are on a round surface (e.g. the earth) so you can view them on a flat surface (e.g. your computer screens or a paper map)."

In [ ]:
world_arctic = world.to_crs('epsg:3031')

##### Did it work?

In [ ]:
world_arctic.crs

##### 🎉 🎉 🎉

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(15,15))
world.plot(ax = ax[0], color='0.8', edgecolor='none')
world_arctic.plot(ax = ax[1], color='0.8', edgecolor='none')

ax[0].set_title('EPSG:4326')
ax[1].set_title('EPSG:3031')

**If we zoom in on Anatarctica, we can see how big a difference these projections make:**

In [ ]:
antarctica_4326 = world[world['name']=='Antarctica']
antarctica_3031 = world_arctic[world_arctic['name']=='Antarctica']

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(15,15))
antarctica_4326.plot(ax = ax[0], color='0.8', edgecolor='none')
antarctica_3031.plot(ax = ax[1], color='0.8', edgecolor='none')

ax[0].set_title('EPSG:4326')
ax[1].set_title('EPSG:3031')

Ahhh Antarctica 🐧

Stereographic projections are common for mapping in polar regions.  A lot of legacy data products for both Greenland and Antarctica use polar stereographic projections. Some other polar products, such as NSIDC EASE/EASE-2 grids, are in _equal-area_ projections.  

```{warning}
Stereographic projections are _conformal_, preserving angles but not distances or areas.  _Equal-area_ map projection cannot be conformal, nor can a conformal map projection be equal-area.  
```

## Reproject other data

Often you have other data or contextual information that you need to get into your data's CRS to visualize. For example, you know McMurdo Research Station is at (-77.846 S, 166.676 E). How to we plot this with out polygon? 

**pyproj transform** objects can be used to change the Coordinate Reference System of arrays

```{important}
Note that most Python libraries do NOT check to make sure your data shares the same CRS. Plotting libraries are not "CRS aware" and will happily plot things in incorrect positions. It is up to you to make sure your positions are accurate.
```

In [ ]:
crs4326 = pyproj.CRS.from_epsg(4326)    # WGS84
crs3031 = pyproj.CRS.from_epsg(3031)    # Antarctic Polar Stereographic
transformer = pyproj.Transformer.from_crs(crs4326, crs3031, always_xy=True)

McMurdo = (166.676, -77.846)
McMurdo_3031 = transformer.transform(*McMurdo)

fig, ax = plt.subplots()
antarctica_3031.plot(ax=ax, color='0.8', edgecolor='none')
ax.plot(McMurdo_3031[0], McMurdo_3031[1], 'ko')
ax.set_title(f'McMurdo Station {McMurdo}');

**Q: Are you more familiar with using vector or raster data?**

**Q: Do you more often use GIS software or command-line tools?**

## ICESat-2

ICESat-2 elevations are in reference to the WGS84 (G1150) ellipsoid. 

```{important}
Recall above we saw EPSG:4326 used "Datum: World Geodetic System 1984 ensemble", which is common for older or low-accuracy datasets. There are different "realizations" of the WGS84 ellipsoid. The accuracy of your positioning improves when the specific realization, rather than the ensemble, is used. [Read more here](https://confluence.qps.nl/qinsy/latest/en/world-geodetic-system-1984-wgs84-182618391.html#id-.WorldGeodeticSystem1984(WGS84)v9.1-WGS84realizations)
```

ICESat-2 data products also include geoid heights from the [EGM2008 geoid](https://www.usna.edu/Users/oceano/pguth/md_help/html/egm96.htm).  Different ground-based, airborne or satellite-derived elevations may use a separate datum entirely! 

```{important}
_Elevations have to be in the same reference frame when comparing heights_!
```

Different datums have different purposes.  Heights above mean sea level are needed for ocean and sea ice heights, and are also commonly used for terrestrial mapping (e.g. as elevations of mountains).  Ellipsoidal heights are commonly used for estimating land ice height change.

### Download ATL06 to a temporary location and open

Note: We just picked a file from random from https://nsidc.org/data/atl06 , we'll extract some elevations from this file and compare against a gridded digital elevation model.

In [ ]:
# NOTE: this requires that you have a ~/.netrc file!
url = 'https://n5eil01u.ecs.nsidc.org/DP7/ATLAS/ATL06.005/2021.12.22/ATL06_20211222214540_00091412_005_01.h5'
filename = 'ATL06_20211222214540_00091412_005_01.h5'

!wget -nc --continue --quiet {url}  -O /tmp/{filename}

In [ ]:
import xarray as xr
ds = xr.open_dataset('/tmp/ATL06_20211222214540_00091412_005_01.h5', group='gt1l/land_ice_segments')
ds

In [ ]:
# For simplicity take first 1000 points into a Geopandas Geodataframe
df = ds.isel(delta_time=slice(0,1000)).to_dataframe()
# NOTE: that the CRS is not propogated by xarray from the HDF5 metadata, so we have to assign it again
gf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs='epsg:7661')
gf.head()

In [ ]:
# Excercise?! add these points to our earlier plot with McMurdo, color by elevation (h_li)
gf.plot.scatter(x='segment_id',y='h_li')

In [ ]:
gf.plot.scatter(x='longitude', y='latitude', c='h_li');

In [ ]:
minx, miny, maxx, maxy = gf.cascaded_union.envelope.bounds
minx, miny, maxx, maxy

In [ ]:
ulx, lry, lrx, uly = gf.cascaded_union.envelope.bounds
print('lower right', lrx, lry)
print('upper left', ulx, uly)

In [ ]:
# Grab Copernicus DEM data for the same area from opentopography
# https://portal.opentopography.org/raster?opentopoID=OTSDEM.032021.4326.3

# Unfortunately, data files alone do not always include the full CRS information! For example, reading the documentation about copernicus
# DEM we see `Vertical Coordinates: EGM2008 [EPSG: 3855]`. These elevations are "geoid" heights. With respect to the EGM2008 model.

# CHECK_DISK_FREE_SPACE=FALSE

!GDAL_DISABLE_READDIR_ON_OPEN=EMPTY_DIR gdal_translate -projwin {ulx} {uly} {lrx} {lry} /vsicurl/https://opentopography.s3.sdsc.edu/raster/COP30/COP30_hh.vrt /tmp/COP30_subset.tif

In [ ]:
!gdalinfo /tmp/COP30_subset.tif

In [ ]:
daC = rioxarray.open_rasterio('https://opentopography.s3.sdsc.edu/raster/COP30/COP30_hh.vrt', 
                              #mask_and_scale=True, 
                              chunks=True)


# Crop by Bounding box of all the ATL06 points
minx, miny, maxx, maxy = gf.cascaded_union.envelope.bounds
daC = daC.rio.clip_box(minx, miny, maxx, maxy)
daC

In [ ]:
import hvplot.xarray
import hvplot.pandas

In [ ]:
gf.hvplot.points(c='h_li', coastline=True, tiles=True, cmap='viridis', data_aspect=0.1)

In [ ]:
img = daC.hvplot.image(rasterize=True, data_aspect=1, cmap='viridis')
points = gf.hvplot

## Geospatial Data Abstraction Library (GDAL)


The [Geospatial Data Abstraction Library (GDAL/OGR)](https://gdal.org/) is a powerful piece of software. 

It can read, write and query a wide variety of raster and vector geospatial data formats, transform the coordinate system of images, and manipulate other forms of geospatial data.

It is the backbone of a _large_ suite of geospatial libraries and programs.

There are a number of wrapper libraries (e.g. [rasterio](https://rasterio.readthedocs.io/), [rioxarray](https://corteva.github.io/rioxarray), [shapely](https://shapely.readthedocs.io/), [fiona](https://fiona.readthedocs.io/)) that provide more user-friendly interfaces with GDAL functionality.

Similar to pyproj CRS objects, GDAL `SpatialReference` functions can provide a lot of information about a particular Coordinate Reference System

GDAL command-line utilities also allows for quick inspection of Coordinate Reference Systems (CRS) and geospatial data files

In [ ]:
! gdalsrsinfo EPSG:3031

<span style="color:orange"> Now let's read in a DEM that overlaps with our IS2 data: ...</span>
* Read in with rasterio and display the crs
* Reproject raster
* Plot the reprojected data within the antarctica polygon to show they line up?
* Plot the ATL06 over the DEM to show they line up

### Let's look back at our CRS descriptions:

In [ ]:
world.crs

In [ ]:
world_arctic.crs

** We can see that the projections are different, but the Datums are the same (the WGS84 ellipsoid). **

**Q: So what is a Datum?**

Reference Systems and Datums
----------------------------
Coordinates are defined to be in _reference_ to the origins of the coordinate system
- Horizontally, coordinates are in reference to the Equator and the Prime Meridian
- Vertically, heights are in reference to a [_datum_](https://vdatum.noaa.gov/docs/datums.html)

Two common vertical datums are the reference _ellipsoid_ and the reference _geoid_.

What are they and what is the difference?
- To ﬁrst approximation, the Earth is a sphere (🐄) with a radius of 6371 kilometers.
- To a better approximation, the Earth is a slightly flattened ellipsoid with the polar axis 22 kilometers shorter than the equatorial axis.
- To an even better approximation, the Earth's shape can be described using a reference geoid, which undulates 10s of meters above and below the reference ellipsoid. The difference in height between the ellipsoid and the geoid are known as _geoid heights_.

The **geoid** is an _equipotential surface_, perpendicular to the force of gravity at all points and with a constant geopotential. Reference ellipsoids and geoids are both created to largely coincide with mean sea level if the oceans were at rest.

An **ellipsoid** can be considered a _simplification of a geoid_.

![Derived from the International Centre for Global Earth Models (ICGEM)](../../img/EGM2008.png)

[PROJ hosts grids for shifting both the horizontal and vertical datum](https://cdn.proj.org/), such as gridded [EGM2008 geoid height values](https://cdn.proj.org/us_nga_egm08_25.tif)

Additional geoid height values can be calculated at the [International Centre for Global Earth Models (ICGEM)](http://icgem.gfz-potsdam.de/home)

## Summary

Congrats! We've made it through the tutorial. You have successfully reprojected geospatial vector data and raster imagery. We looked specifically at ICESat-2 ATLO6 elevation data and compared it against a global digitial elevation dataset. Spend some time looking at data from locations other than antarctica requiring different CRS projections, and explore the resources below to learn more!

## References

There are entire graduate-level courses and great educational material to learn more about these topics. We recommend checking out:

* [PennState GPS & GNSS for Geospatial Professionals](https://www.e-education.psu.edu/geog862/node/1794)
* [UW Geospatial Data Analaysis with Python](https://uwgda-jupyterbook.readthedocs.io/en/latest/modules/04_Vector1_Geopandas_CRS_Proj/04_Vector1_Geopandas_CRS_Proj_prep.html)
* [Qinsy Geodetic Documentation](https://confluence.qps.nl/qinsy/latest/en/how-to-geodetic-items-235803916.html)